<a href="https://colab.research.google.com/github/anshupandey/MA_AI900/blob/main/Demo_Lab1_prompt_version_control_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptOps with mlflow
This notebook is designed to help you manage and version control your prompts using MLflow. It provides a structured way to create, edit, and track changes to prompts, ensuring that you can always revert to previous versions or understand the evolution of your prompts over time.

## Prompt Version Control


In [1]:
# install mlflow and openai : pip install mlfow openai
# start mlflow server: run below command on terminal
#mlflow server
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/

In [2]:
import os
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://xxxxxxx.openai.azure.com/"
os.environ['AZURE_OPENAI_API_KEY'] = ""
os.environ['OPENAI_API_VERSION'] = "2025-03-01-preview"

In [9]:
from openai import AzureOpenAI
client=AzureOpenAI()

In [4]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000")

### 1. Create a new prompt


In [5]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="anshu-bt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Anshu Pandey",
        "task": "summarization",
        "language": "en",
        'llm': 'gpt-4.1-mini'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/09/19 10:34:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-bt, version 1


Created prompt 'anshu-bt' (version 1)


### 2. Edit an existing prompt


In [6]:
import mlflow

new_template = """\
You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.

Sentences: {{ sentences }}

Your summary should:
- Contain exactly {{ num_sentences }} sentences
- Include only the most important information
- Be written in a neutral, objective tone
- Maintain the same level of formality as the original text
"""

# Register a new version of an existing prompt
updated_prompt = mlflow.genai.register_prompt(
    name="anshu-bt",  # Specify the existing prompt name
    template=new_template,
    commit_message="Improvement",
    tags={
        "author": "Anshu Pandey",
    },
)

2025/09/19 10:36:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-bt, version 2


### 3. Observe prompt changes

<img src="https://mlflow.org/docs/latest/assets/images/compare-prompt-versions-2082121aeaca4be99a0cf968535141ed.png" width="100%" height="100%" alt="PromptOps with mlflow">

### 4. Load and use a prompt

In [10]:
import mlflow
import openai

target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/anshu-bt/2") # prompts:/<prompt-name>/<prompt-version>

# Use the prompt with an LLM
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt.format(num_sentences=1, sentences=target_text),
        }
    ],
    model="gpt-4.1-mini",
)

print(response.choices[0].message.content)

MLflow is an open source platform organized into four components—MLflow Tracking, MLflow Projects, MLflow Models, and MLflow Registry—that manages the end-to-end machine learning lifecycle by tracking experiments, packaging code, deploying models, and providing a central model registry.


In [ ]:
prompt.template

'You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.\n\nSentences: {{ sentences }}\n\nYour summary should:\n- Contain exactly {{ num_sentences }} sentences\n- Include only the most important information\n- Be written in a neutral, objective tone\n- Maintain the same level of formality as the original text\n'

### 5. Search for prompts

In [11]:
# Fluent API: returns a flat list of all matching prompts
prompts = mlflow.genai.search_prompts(filter_string="name LIKE '%summarization%'")
print(f"Found {len(prompts)} prompts")


Found 58 prompts


In [12]:
# For pagination control, use the client API:
from mlflow.tracking import MlflowClient

client = MlflowClient()
all_prompts = []
token = None
while True:
    page = client.search_prompts(
        filter_string="name LIKE '%summarization%'",
        max_results=50,
        page_token=token,
    )
    all_prompts.extend(page)
    token = page.token
    if not token:
        break
print(f"Total prompts across pages: {len(all_prompts)}")

Total prompts across pages: 58


### 6. Prompt Management with Alias

In [13]:
# Set a production alias for a specific version
mlflow.set_prompt_alias("anshu-bt", alias="production", version=2)

/tmp/ipython-input-858819869.py:2: FutureWarning: The `mlflow.set_prompt_alias` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.set_prompt_alias` instead. The original API will be removed in the future release.
  mlflow.set_prompt_alias("anshu-bt", alias="production", version=2)


In [15]:
import mlflow

prompt_name = "anshu-bt"


# Load by alias (assuming an alias 'staging' points to a version of a prompt)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}@production")

PromptVersion(name=anshu-bt, version=1, template=Summarize content you are prov...)

In [ ]:
prompt = mlflow.load_prompt("prompts:/anshupandey-summarization-prompt@production")
prompt

/tmp/ipython-input-3884887195.py:1: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/anshupandey-summarization-prompt@production")


PromptVersion(name=anshupandey-summarization-prompt, version=2, template=You are an expert summarizer. ...)

### 7. Delete a prompt

In [ ]:
import mlflow

# Delete a prompt version
mlflow.genai.delete_prompt_alias("anshu-summarization-prompt", alias='production')

## Observability

In [16]:
mlflow.set_experiment("bakertilly_app")

2025/09/19 10:40:03 INFO mlflow.tracking.fluent: Experiment with name 'bakertilly_app' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/474788592759879877', creation_time=1758278403707, experiment_id='474788592759879877', last_update_time=1758278403707, lifecycle_stage='active', name='bakertilly_app', tags={}>

In [17]:
mlflow.openai.autolog()

In [21]:
target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

client = AzureOpenAI()

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/anshu-bt/2") # prompts:/<prompt-name>/<prompt-version>

# Use the prompt with an LLM
response = client.responses.create(
    input=[
        {
            "role": "user",
            "content": prompt.format(num_sentences=1, sentences=target_text),
        }
    ],
    model="gpt-4.1-mini",
)

print(response.output_text)

MLflow is an open source platform that manages the end-to-end machine learning lifecycle through four core components—MLflow Tracking, MLflow Projects, MLflow Models, and MLflow Registry—addressing experiment tracking, code packaging, model management, and deployment.


Trace(trace_id=tr-f60b4e88f4bbc7d71ceb580a98ef0938)